In [114]:
import numpy as np
import pandas as pd
from torch.nn import MSELoss
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from IPython.display import display, Math, Latex

In [115]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [116]:
def descriptive_length_of_fraction(numerator, denominator):
    return torch.log2((1 + torch.abs(torch.tensor(denominator))) * torch.abs(numerator))


def logplus(number):
    return torch.log2(1 + number**2) / 2


def descriptive_length_of_real_number(real_number, precision_floor=1e-8):
    return logplus(real_number / torch.tensor(precision_floor))

In [170]:
def FormFractionRepresentation(fraction: torch.tensor) -> str:
    if fraction[1].item() != 1:
        return r"\frac{" + str(int(fraction[0].item())) + "}{" + str(int(fraction[1].item())) + "}"
    return str(int(fraction[0].item()))


def FormReal(number: torch.tensor) -> str:
    return "{:.3}".format(number.item())


def AddRationalInName(name: str) -> str:
    if 'lambda' in name:
        position = name.find('lambda')
    else:
        position = name.find('power')
    return name[:position] + "rational_" + name[position:]

In [118]:
def info(formula):
    print("depth: {}, number of variables: {}, total parameters: {}".format(
        formula.depth, formula.num_variables, len(formula.parameters)))
    
def PrintFormula(formula, mode="slow"):
#     info(network)
    if mode == "slow":
        display(Math(str(formula)))   
    else:
        print(formula)
        

In [144]:
class RecursiveFormula(nn.Module):
    """
    Class used for representing formulas
    
    Attributes:
        depth
        num_variables
        powers
        lambdas - list of linear coefficients
        subformulas - list of subformulas of smaller depth, which are used for computing
        parameters - list of all learnable parameters of formula (real numbers)
        rational_values - if during simplification it turns out that some parameter is actually a rational number, 
                            its value is stored here as a tuple (numerator, denominator), otherwise there is None
        last_subformula - additional subformula, which is not multiplied by variable
    """
    
    def __init__(self, depth=0, num_variables=1):
        super(RecursiveFormula, self).__init__()
        self.depth = depth
        self.num_variables = num_variables
        self.num_parameters = 0
        self.powers = []
        self.lambdas = []
        self.subformulas = nn.ModuleList()
#         self.parameters = []
        # When depth is zero, formula is just a real number
        if depth == 0:
            new_lambda = nn.Parameter((2 * torch.randn((1, 1)))).requires_grad_(True)
            self.lambdas.append(new_lambda)
            self.register_parameter("lambda_0", new_lambda)
            new_rational_lambda = nn.Parameter(torch.tensor([0., 0.])).requires_grad_(False)
            self.register_parameter("rational_lambda_0", new_rational_lambda)
            self.num_parameters += 1
#             self.parameters.append(new_lambda)
        else:
            for i in range(self.num_variables):
                # When depth is 1, we do not need to create subformulas, since they would be just real numbers
                if self.depth != 1:
                    subformula = RecursiveFormula(self.depth - 1, self.num_variables)
                    self.subformulas.append(subformula)
                    self.num_parameters += subformula.num_parameters
#                     self.parameters.extend(subformula.parameters)                    
                new_lambda = nn.Parameter((2 * torch.randn((1, 1)))).requires_grad_(True)
                new_power = nn.Parameter((2 * torch.randn((1, 1)))).requires_grad_(True)
#                 new_rational_lambda = nn.Parameter(torch.round(new_lambda)).requires_grad_(False)
#                 new_rational_power = nn.Parameter(torch.round(new_power)).requires_grad_(False)
                new_rational_lambda = nn.Parameter(torch.tensor([0., 0.])).requires_grad_(False)
                new_rational_power = nn.Parameter(torch.tensor([0., 0.])).requires_grad_(False)
                self.register_parameter("lambda_{}".format(i), new_lambda)
                self.register_parameter("power_{}".format(i), new_power)
                self.register_parameter("rational_lambda_{}".format(i), new_rational_lambda)
                self.register_parameter("rational_power_{}".format(i), new_rational_power)
                self.lambdas.append(new_lambda)
                self.powers.append(new_power)
                self.num_parameters += 2
#                 self.parameters.extend([new_lambda, new_power])
            self.last_subformula = RecursiveFormula(self.depth - 1, self.num_variables)
            self.num_parameters += self.last_subformula.num_parameters
#             self.parameters.extend(self.last_subformula.parameters)
        # Before simplification we assume that all parameters are real
        self.rational_lambdas = [None for lambd in self.lambdas]
        self.rational_powers = [None for power in self.powers]
        
                                    
    def forward(self, x):
        """
        Iterate over subformulas, recursively computing result using results of subformulas
        """
        # When depth is 0, we just return the corresponding number
        if self.depth == 0:
            return self.lambdas[0].repeat(x.shape[0], 1)
        
        ans = torch.zeros(x.shape[0], 1)
        for i in range(self.num_variables):
            x_powered = torch.t(x[:, i]**self.powers[i])
            subformula_result = torch.ones((x.shape[0], 1))
            # When depth is 1, we do not need to compute subformulas
            if self.depth != 1:
                subformula_result = self.subformulas[i](x)
            assert subformula_result.shape == (x.shape[0], 1)
            assert x_powered.shape == (x.shape[0], 1)
            ans += self.lambdas[i] * x_powered * subformula_result           
        ans += self.last_subformula(x)
        return ans
    
    def simplify(self, X_val, y_val, max_denominator=1, inplace=False):
        simplified_version = copy.deepcopy(self)  
        simplified_state_dict = simplified_version.state_dict()
        
        # Iterate over all parameters, try to substitute them with some rational number.
        for key, value in self.state_dict().items():
            if "rational" not in key: # We do not simplify rational parameters - they will be the result of simplification
                simplified_version_for_iteration = copy.deepcopy(simplified_version)
                simplified_state_dict_for_iteration = simplified_version_for_iteration.state_dict()
                y_predict = simplified_version(X_val)
                loss = MSELoss()(y_val, y_predict)
                descriptive_length_of_loss = descriptive_length_of_real_number(loss)
                descriptive_length_of_existing_parameter = descriptive_length_of_real_number(value)

                # Iterate over all possible denominators
                for possible_denominator in range(1, max_denominator + 1):
                    print("trying denominator", possible_denominator)
                    simplified_parameter_numerator = torch.round(value * possible_denominator)
                    simplified_state_dict_for_iteration[key] = simplified_parameter_numerator / possible_denominator
                    simplified_version_for_iteration.load_state_dict(simplified_state_dict_for_iteration)
                    descriptive_length_of_simplified_parameter = descriptive_length_of_fraction(simplified_parameter_numerator, possible_denominator)
                    print(simplified_parameter_numerator, possible_denominator)
                    y_predict_simplified = simplified_version_for_iteration(X_val)
                    loss_of_simplified_model = MSELoss()(y_val, y_predict_simplified)
                    descriptive_length_of_loss_of_simplified_model = descriptive_length_of_real_number(loss_of_simplified_model)                
                    # If the descriptive length did not improve, revert the change.
                    print("descriptive_length_of_loss_of_simplified_model", descriptive_length_of_loss_of_simplified_model)
                    print("descriptive_length_of_simplified_parameter", descriptive_length_of_simplified_parameter)
                    print("descriptive_length_of_loss", descriptive_length_of_loss)
                    print("descriptive_length_of_existing_parameter", descriptive_length_of_existing_parameter)

                    if descriptive_length_of_loss_of_simplified_model + descriptive_length_of_simplified_parameter > descriptive_length_of_loss + descriptive_length_of_existing_parameter:
                        simplified_version_for_iteration.load_state_dict(simplified_state_dict)
                    else:
                        # If we are successful, we update everything
                        print(key)
                        simplified_state_dict[AddRationalInName(key)] = torch.tensor([simplified_parameter_numerator, possible_denominator])
                        simplified_version.load_state_dict(simplified_state_dict)
                        simplified_version_for_iteration = copy.deepcopy(simplified_version)
                        simplified_state_dict_for_iteration = simplified_version_for_iteration.state_dict()

                simplified_state_dict = simplified_state_dict_for_iteration
                simplified_version.load_state_dict(simplified_state_dict)
        
        if inplace:
            self = copy.deepcopy(simplified_version)
        else:
            return simplified_version      
    
    def get_lambda(self, i):
        return self.state_dict()['lambda_{}'.format(i)]
    
    def get_rational_lambda(self, i):
        return self.state_dict()['rational_lambda_{}'.format(i)]
    
    def get_power(self, i):
        return self.state_dict()['power_{}'.format(i)]
    
    def get_rational_power(self, i):
        return self.state_dict()['rational_power_{}'.format(i)]
    
    def __repr__(self):
        """
        Return tex-style string, recursively combining result from representation of subformulas
        """
        if self.depth == 0:
            if self.get_rational_lambda(0)[1] > 0: # if it is equal to 0, it means that there is no rational value
                return FormFractionRepresentation(self.get_rational_lambda(0))            
            return FormReal(self.get_lambda(0))
        
        ans = ["("]
        for i in range(self.num_variables):
            # First we add lambda
            if i != 0 and self.get_lambda(i) > 0:
                ans.append(" + ")
            if self.get_rational_lambda(i)[1] > 0:
                ans.append(FormFractionRepresentation(self.get_rational_lambda(i)))
            else:
                ans.append(FormReal(self.get_lambda(i)))   
            # Then we add variable and its power
            ans.append("x_{}^".format(i + 1) + "{")
            if self.get_rational_power(i)[1] > 0:
                ans.append(FormFractionRepresentation(self.get_rational_power(i)))
            else:
                ans.append(FormReal(self.get_power(i)))  
            ans += "}"    
            # Then we add the corresponding subformula
            if self.depth != 1:
                ans.append(str(self.subformulas[i]))
        if self.last_subformula.lambdas[0] > 0:        
            ans.append(" + ")
        ans.append(str(self.last_subformula))
        ans.append(")")
        ans = ''.join(ans)
        return ans

In [141]:
def LearnFormula(X, y, n_init=1, max_iter=2000, depth=2, verbose=2, max_epochs_without_improvement=500,
                minimal_acceptable_improvement=3e-6):
    best_model = RecursiveFormula(depth, X.shape[1])
    best_loss = 1e10
    for initiation in range(n_init):
    #     torch.random.manual_seed(seed)
        formula = RecursiveFormula(depth, X.shape[1])
        # create your optimizer
        optimizer = optim.Rprop(formula.parameters(), lr=1e-3)
        criterion = nn.MSELoss()
        epochs_without_improvement = 0
        epoch = 0
        optimizer.zero_grad()
        output = formula(X)
        previous_loss = criterion(output, y).item() 
        while epoch < max_iter and epochs_without_improvement < max_epochs_without_improvement:
            optimizer.zero_grad()
            output = formula(X)
            loss = criterion(output, y) 
            loss.backward()
            if verbose and (epoch + 1) % 500 == 0:
                print("Epoch {}, current loss {:.3}, current formula ".format(epoch + 1, loss.item()), end='')
                PrintFormula(formula, "fast")       
            optimizer.step()  
            epoch += 1
            if torch.abs(previous_loss - loss) < minimal_acceptable_improvement:
                epochs_without_improvement += 1
#                 print(torch.abs(previous_loss - loss))
            else:
                epochs_without_improvement = 0
            previous_loss = loss.item()
        if loss < best_loss:
            best_loss = loss
            best_formula = formula
    return best_formula

In [129]:
X1 = torch.rand(100, 1) * 10
y1 = 2.5 * X1**2 + 3

X2 = torch.rand(100, 2) * 10
y2 = 2.5 * X2[:, 0]**2 + 0.333 * X2[:, 1]**0.5 + 3    
# y = 1.2 * X[:, 0]**2.1 * X[:, 2] + 2.5 * X[:, 1]**(-3) + 1/2 * X[:, 2]**0.3333 * X[:, 1]**(-4)

In [148]:
formula = LearnFormula(X1, y1)

Epoch 500, current loss 0.167, current formula (1.63x_1^{0.88}(3.05x_1^{-0.464}-3.76) + (3.9x_1^{1.86} + 3.14))
Epoch 1000, current loss 0.0695, current formula (1.63x_1^{0.977}(3.44x_1^{-0.43}-3.75) + (3.9x_1^{1.87} + 2.46))
Epoch 1500, current loss 0.0378, current formula (1.63x_1^{1.03}(3.68x_1^{-0.429}-3.75) + (3.9x_1^{1.87} + 2.05))
Epoch 2000, current loss 0.0241, current formula (1.63x_1^{1.06}(3.82x_1^{-0.429}-3.75) + (3.9x_1^{1.88} + 1.77))


In [149]:
simplified_formula = formula.simplify(X1, y1, inplace=False)

trying denominator 1
tensor([[2.]]) 1
descriptive_length_of_loss_of_simplified_model tensor(31.4934, grad_fn=<DivBackward0>)
descriptive_length_of_simplified_parameter tensor([[2.]])
descriptive_length_of_loss tensor(21.1975, grad_fn=<DivBackward0>)
descriptive_length_of_existing_parameter tensor([[27.2779]])
lambda_0
trying denominator 1
tensor([[1.]]) 1
descriptive_length_of_loss_of_simplified_model tensor(29.4192, grad_fn=<DivBackward0>)
descriptive_length_of_simplified_parameter tensor([[1.]])
descriptive_length_of_loss tensor(21.1975, grad_fn=<DivBackward0>)
descriptive_length_of_existing_parameter tensor([[26.6554]])
power_0
trying denominator 1
tensor([[4.]]) 1
descriptive_length_of_loss_of_simplified_model tensor(26.2320, grad_fn=<DivBackward0>)
descriptive_length_of_simplified_parameter tensor([[3.]])
descriptive_length_of_loss tensor(21.1975, grad_fn=<DivBackward0>)
descriptive_length_of_existing_parameter tensor([[28.5086]])
subformulas.0.lambda_0
trying denominator 1
tensor

In [172]:
PrintFormula(simplified_formula)

<IPython.core.display.Math object>

In [164]:
a = r""
a += r"2 \frac{1}{3}"
print(a)
a += '123'
print(a)


2 \frac{1}{3}
2 \frac{1}{3}123


In [ ]:
a = RecursiveFormula(3, 3)

In [ ]:
old_dict = a.state_dict()
old_dict['lambda_0'] = torch.tensor([[179]])
a.load_state_dict(old_dict)

In [ ]:
a.state_dict()

In [ ]:
PrintFormula(LearnFormula(X2, y2))

In [ ]:
MSELoss()(y1, y1)

In [ ]:
for i in formula.parameters():
    print(i)